 ## Calculate overlap
Generate the DICE coefficient for each segmentation using the *EDIT1 file as reference. 

In [2]:
# Import packages
import ants
import numpy as np
import pandas as pd
from pathlib import Path
import logging
import os
import subprocess

PATH_TO_DATA = Path('test-data')

In [3]:
# Instantiate a logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger('root')

In [43]:
def refine_list(lst):
    result_list = str(lst).split(',')
    result_list[-1] = result_list[-1].replace("\\n", "").replace("'", "")
    result_list = [float(result_list[4]), float(result_list[5])]
#     result_list = [float(e) for e in result_list]
    
    return(result_list)
    

Before running the next step, I reoriented the reference segmentations using `organizeAndReorient.sh`.

In [52]:
# Run the DICE calculations
df_list = [] # each output is a pandas DataFrame that we'll combine at the end based on this list
for subj in PATH_TO_DATA.glob('*S*'): # e.g., 011_S_6303
    log.info('Processing subject %s', str(subj))
    for ses in subj.glob('20*'): # 2017, 2018, etc.
        log.info('Processing session %s', str(ses))
        
        new_seg_file = str(list(ses.glob("*WMH_seg*"))[0])
        new_10_seg_file = str(list(ses.glob("*WMH_10x_seg*"))[0])
        baseline_seg_file = str(list(ses.glob("*reorient_EDIT1*"))[0])
                
        # copy header from new seg to old seg (only have to do once) to match origins
        cmd = 'c3d {0} {1} -copy-transform -o {1}'.format(new_seg_file, baseline_seg_file)
        log.info(os.system(cmd))
        
        # calculate DICE with c3d
        cmd2="c3d {} {} -overlap 1".format(new_seg_file, baseline_seg_file)
        result = refine_list(subprocess.check_output(cmd2, shell=True))
        cmd3="c3d {} {} -overlap 1".format(new_10_seg_file, baseline_seg_file)
        result10 = refine_list(subprocess.check_output(cmd3, shell=True))
        log.info("{} ; {}".format(result, result10))

        # Do the ANTscalculation for both segmentations
        new_seg = ants.image_read(new_seg_file)  # read the data
        new_10_seg = ants.image_read(new_10_seg_file)
        baseline_seg = ants.image_read(baseline_seg_file)
        
        std_comp = ants.label_overlap_measures(baseline_seg, new_seg) # run the calculation
        x10_comp = ants.label_overlap_measures(baseline_seg, new_10_seg)
        
        # Edit the dataframe a little
        std_comp['c3dDice'] = result[0] # Dice
        x10_comp['c3dDice'] = result10[0] 
        std_comp['c3dJacob'] = result[1] # Jacobian
        x10_comp['c3dJacob'] = result10[1]
        std_comp.insert(0, 'SubjectID', subj.name) # insert subject name in first position
        x10_comp.insert(0, 'SubjectID', subj.name)
        std_comp.insert(1, 'Type', 'normal') # specify which image is which
        x10_comp.insert(1, 'Type', 'scaled10x')
        std_comp=std_comp.iloc[:1] # just get the first row
        x10_comp=x10_comp.iloc[:1]
        
        df_list.append(std_comp) # add to list for concatenation later
        df_list.append(x10_comp)    
        
df = pd.concat(df_list)
df = df.reset_index(drop=True)
df = df.drop('Label',1) # drop superfluous column
log.info('DONE')

2021-09-22 12:29:11,788 INFO Processing subject test-data/027_S_6034
2021-09-22 12:29:11,789 INFO Processing session test-data/027_S_6034/2019-07-24
2021-09-22 12:29:12,474 INFO 0
2021-09-22 12:29:13,401 INFO [0.433333, 0.276596] ; [0.464, 0.302083]
2021-09-22 12:29:14,109 INFO Processing subject test-data/168_S_6426
2021-09-22 12:29:14,110 INFO Processing session test-data/168_S_6426/2018-06-29
2021-09-22 12:29:14,706 INFO 0
2021-09-22 12:29:15,542 INFO [0.7808, 0.640421] ; [0.797701, 0.66348]
2021-09-22 12:29:16,188 INFO Processing subject test-data/027_S_5277
2021-09-22 12:29:16,188 INFO Processing session test-data/027_S_5277/2017-11-30
2021-09-22 12:29:16,848 INFO 0
2021-09-22 12:29:17,801 INFO [0.435734, 0.278555] ; [0.451092, 0.291232]
2021-09-22 12:29:18,516 INFO Processing subject test-data/114_S_6347
2021-09-22 12:29:18,517 INFO Processing session test-data/114_S_6347/2018-05-26
2021-09-22 12:29:19,101 INFO 0
2021-09-22 12:29:19,917 INFO [0.874533, 0.77704] ; [0.869013, 0.768

In [56]:
df.to_csv('overlap_data.csv',index=False)